In [12]:
import pandas as pd
import numpy as np
import subfunc as sf

In [13]:
RAW_DATA = 'data/evras.csv'
POST_PROCESSING_DATA = 'data/post_evras.csv'
SOURCE_DATA = 'source/'
pd.options.display.max_columns = 90
target_columns = ['химшлак последний Al2O3',
                  'химшлак последний CaO',
                  'химшлак последний R',
                  'химшлак последний SiO2']

In [42]:
data = pd.read_csv(RAW_DATA, sep=';', index_col='index')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7041 entries, 0 to 7040
Data columns (total 84 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   nplv                            7041 non-null   object
 1   DT                              7041 non-null   object
 2   МАРКА                           7041 non-null   object
 3   ПРОФИЛЬ                         7041 non-null   object
 4   t вып-обр                       6444 non-null   object
 5   t обработка                     6440 non-null   object
 6   t под током                     6440 non-null   object
 7   t продувка                      6444 non-null   object
 8   ПСН гр.                         5249 non-null   object
 9   чист расход C                   6435 non-null   object
 10  чист расход Cr                  5883 non-null   object
 11  чист расход Mn                  6431 non-null   object
 12  чист расход Si                  6431 non-null   

ферспл FeSi-75                7036
сыпуч  кокс. мелочь (сух.)    7034
сыпуч кварцит                 7028
ферспл FeMo                   7008
ферспл  Ni H1 пласт.          6959
                              ... 
химшлак первый CaO_1           597
ПРОФИЛЬ                          0
МАРКА                            0
DT                               0
nplv                             0
Length: 84, dtype: int64

In [46]:
data.isnull().sum().sort_values(ascending= False).apply(lambda x : x / data.shape[0])[:15]

ферспл FeSi-75                0.999290
сыпуч  кокс. мелочь (сух.)    0.999006
сыпуч кварцит                 0.998154
ферспл FeMo                   0.995313
ферспл  Ni H1 пласт.          0.988354
ферспл FeV азот.              0.945604
расход C пров.                0.897174
ферспл  ферванит              0.855560
расход газ  N2                0.802727
N2 (интенс.)                  0.802727
химшлак первый SiO2_1         0.326516
химшлак последний Al2O3       0.326516
химшлак первый Al2O3_1        0.326516
химшлак последний SiO2        0.326516
сыпуч известь РП              0.322539
dtype: float64

In [15]:
data.describe()

,nplv,DT,МАРКА,ПРОФИЛЬ,t вып-обр,t обработка,t под током,t продувка,ПСН гр.,чист расход C,чист расход Cr,чист расход Mn,чист расход Si,чист расход V,температура первая,температура последняя,Ar (интенс.),N2 (интенс.),эл. энергия (интенс.),произв жидкая сталь,произв количество обработок,произв количество плавок,произв количество плавок (цел),расход газ Ar,расход газ N2,расход C пров.,сыпуч известь РП,сыпуч кварцит,сыпуч кокс пыль УСТК,сыпуч кокс. мелочь (сух.),сыпуч кокс. мелочь КМ1,сыпуч шпат плав.,ферспл CaC2,ферспл FeMo,ферспл FeSi-75,ферспл FeV азот.,ферспл FeV-80,ферспл Mn5Si65Al0.5,ферспл Ni H1 пласт.,ферспл SiMn18,ферспл ферванит,ферспл фх850А,эл. энергия,химсталь первый Al_1,химсталь первый C_1,химсталь первый Cr_1,химсталь первый Cu_1,химсталь первый Mn_1,химсталь первый Mo_1,химсталь первый N_1,химсталь первый Ni_1,химсталь первый P_1,химсталь первый S_1,химсталь первый Si_1,химсталь первый Ti_1,химсталь первый V_1,химсталь последний Al,химсталь последний C,химсталь последний Ca,химсталь последний Cr,химсталь последний Cu,химсталь последний Mn,химсталь последний Mo,химсталь последний N,химсталь последний Ni,химсталь последний P,химсталь последний S,химсталь последний Si,химсталь последний Ti,химсталь последний V,химшлак первый Al2O3_1,химшлак первый CaO_1,химшлак первый FeO_1,химшлак первый MgO_1,химшлак первый MnO_1,химшлак первый R_1,химшлак первый SiO2_1,химшлак последний Al2O3,химшлак последний CaO,химшлак последний FeO,химшлак последний MgO,химшлак последний MnO,химшлак последний R,химшлак последний SiO2
count,7041,7041,7041,7041,6444,6440,6440,6444,5249,6435,5883,6431,6431,6178,6436,6436,6426,1389,6440,6444,6444,6444,6444,6426,1389,724,4770,13,6422,7,6307,5910,6244,33,5,383,6179,6400,82,6398,1017,5883,6440,6444,6444,6444,6444,6444,6443,6443,6444,6444,6444,6444,6444,6444,6444,6444,6218,6444,6444,6444,6444,6442,6444,6444,6444,6444,6444,6444,4742,6444,6444,6443,6444,6444,4742,4742,6444,6444,6443,6444,6444,4742
unique,7041,7041,18,18,82,816,1155,831,4296,6243,3115,6320,6265,4645,154,109,6423,1389,6362,226,3,1,1,5556,1341,593,469,1,307,7,691,197,7,28,2,83,692,399,80,528,199,398,3132,6,417,371,101,404,24,18,115,18,34,384,7,28,20,226,32,206,100,261,24,17,112,19,27,256,5,54,73,298,54,236,152,32,149,70,229,16,192,61,16,116
top,г683863<97=,2071-01-08 09:17:00,Э76ХФ,Р65,"6,0","42,5333333","19,6833333","43,0333333","2,0","0,406765","0,104904","0,132318","0,125824","0,0388151","1571,0","1579,0","9,9717791","13,0398714","13200,0","115,4","1,0","1,0","1,0","13,192","9,016","0,013442","0,063","0,02","0,05","0,498","0,384","0,041","0,02","0,021014","0,05","0,024306","0,050094","0,128","0,046115","0,219","0,025897","0,214","4199,0","0,002","0,435","0,315","0,1","0,672","0,006","0,01","0,071","0,008","0,019","0,449","0,002","0,002","0,003","0,762","0,0002","0,418","0,103","0,789","0,006","0,012","0,071","0,009","0,015","0,566","0,003","0,037","3,7","53,4","0,4","11,3","0,11","2,2","21,7","4,4","57,6","0,5","8,7","0,08","2,2","26,0"
freq,1,1,4953,6427,227,53,32,56,526,3,17,3,2,9,134,282,2,1,4,124,6391,6444,6444,5,3,4,87,13,5873,1,34,250,6137,3,4,14,81,61,2,45,23,60,11,4656,62,91,236,73,1783,1751,308,1410,904,105,3727,2929,4526,166,1816,660,240,353,1590,1305,324,1363,1076,229,4764,1793,263,58,1056,96,470,915,100,243,102,1235,88,723,1483,142


In [16]:
data['DT'] = sf.date_to_interval(data['DT'])

In [47]:
dropped_data = sf.drop_columns(data,0.65)

Удаленные колонки: N2 (интенс.), расход газ  N2, расход C пров., сыпуч кварцит, сыпуч  кокс. мелочь (сух.), ферспл FeMo, ферспл FeSi-75, ферспл FeV азот., ферспл  Ni H1 пласт., ферспл  ферванит


In [51]:
dropped_data.isnull().sum().sort_values(ascending= False).apply(lambda x : x / data.shape[0])

химшлак последний SiO2     0.326516
химшлак последний Al2O3    0.326516
химшлак первый Al2O3_1     0.326516
химшлак первый SiO2_1      0.326516
сыпуч известь РП           0.322539
                             ...   
химсталь последний Cr      0.084789
ПРОФИЛЬ                    0.000000
МАРКА                      0.000000
DT                         0.000000
nplv                       0.000000
Length: 74, dtype: float64

In [19]:
u, count = np.unique(data['МАРКА'], return_counts=True)
count_sort_ind = np.argsort(-count)
top_mark = u[count_sort_ind][0]
top_mark

'Э76ХФ'

In [20]:
mark_data = data[data['МАРКА'] == top_mark]

In [21]:
mark_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4953 entries, 0 to 7026
Data columns (total 84 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   nplv                            4953 non-null   object
 1   DT                              4953 non-null   object
 2   МАРКА                           4953 non-null   object
 3   ПРОФИЛЬ                         4953 non-null   object
 4   t вып-обр                       4432 non-null   object
 5   t обработка                     4429 non-null   object
 6   t под током                     4429 non-null   object
 7   t продувка                      4432 non-null   object
 8   ПСН гр.                         3590 non-null   object
 9   чист расход C                   4426 non-null   object
 10  чист расход Cr                  4417 non-null   object
 11  чист расход Mn                  4423 non-null   object
 12  чист расход Si                  4423 non-null   

In [22]:
mark_data.head()

NameError: name 'nan_columns' is not defined

In [ ]:
mark_data.fillna(0, inplace=True)
mark_data.info().s

In [ ]:
mark_data = mark_data.apply(lambda x: x.apply(str).str.replace(',', '.') )
mark_data = mark_data.apply(lambda x: pd.to_numeric(x))

In [ ]:
mark_data.head()

In [ ]:
mark_data.to_csv(POST_PROCESSING_DATA, sep='\t', index=False)

In [ ]:
mark_data.to_csv(POST_PROCESSING_DATA, sep='\t', index=False)

In [ ]:
mark_data = data[data['МАРКА'] == top_mark]


In [ ]:
mark_data = mark_data.drop(columns=nan_columns)
mark_data.head()

In [ ]:
mark_data.fillna(0, inplace=True)
mark_data.info()

In [ ]:
mark_data = mark_data.apply(lambda x: x.apply(str).str.replace(',', '.') )
mark_data = mark_data.apply(lambda x: pd.to_numeric(x))

In [ ]:
mark_data.head()

In [ ]:
mark_data.to_csv(POST_PROCESSING_DATA, sep='\t', index=False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


plt.figure(figsize=(7, 7))
sns.distplot(mark_data['химшлак последний Al2O3'], color='g', bins=100, hist_kws={'alpha': 0.4}, vertical=True)
#plt.show()
plt.savefig(f'{SOURCE_DATA}target_range_1.png')

In [ ]:
plt.figure(figsize=(7, 7))
sns.distplot(mark_data['химшлак последний CaO'], color='r', bins=100, hist_kws={'alpha': 0.4}, vertical=True)
#plt.show()
plt.savefig(f'{SOURCE_DATA}target_range_2.png')

In [ ]:
plt.figure(figsize=(7, 7))
sns.distplot(mark_data['химшлак последний R'], color='g', bins=100, hist_kws={'alpha': 0.4}, vertical=True)
plt.show()
plt.savefig(f'{SOURCE_DATA}target_range_3.png')

In [ ]:
plt.figure(figsize=(7, 7))
sns.distplot(mark_data['химшлак последний SiO2'], color='r', bins=100, hist_kws={'alpha': 0.4}, vertical=True)
plt.show()
plt.savefig(f'{SOURCE_DATA}target_range_4.png')

In [ ]:
corr_data = mark_data.drop(columns=target_columns).corr()

plt.figure(figsize=(52, 50))

sns.heatmap(corr_data[(corr_data >= 0.5) | (corr_data <= -0.4)],
            cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True)
plt.savefig(f'{SOURCE_DATA}correletion_map.png')

In [ ]:
mark_data.to_csv(POST_PROCESSING_DATA, sep='\t', index=False)

In [ ]:
mark_data.fillna(0, inplace=True)
mark_data.info()

In [ ]:
mark_data = mark_data.apply(lambda x: x.apply(str).str.replace(',', '.') )
mark_data = mark_data.apply(lambda x: pd.to_numeric(x))

In [ ]:
mark_data.head()

In [ ]:
mark_data.to_csv(POST_PROCESSING_DATA, sep='\t', index=False)